<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/master/Rapid_Mix_Flocculation_Mechanical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rapid-Mix and Flocculation Mechanical

Authors: Ariel Ben-Sorek, Lynn Li, and Allison Tran

[/Monroe/]: # (See my email for general comments.)

In [7]:
!pip install aguaclara
!pip install molmass

In [0]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/MWSteams/4590FSP/master/Fairmont.py
import Fairmont as fmt
import math

--2020-04-29 14:06:38--  https://raw.githubusercontent.com/MWSteams/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2538 (2.5K) [text/plain]
Saving to: ‘Fairmont.py’

Fairmont.py         100%[===================>]   2.48K  --.-KB/s    in 0s      

2020-04-29 14:06:39 (43.9 MB/s) - ‘Fairmont.py’ saved [2538/2538]



# Task Definition

Our task was to evaluate both processes of mechanical rapid mix and mechanical flocculation, in order to determine the most effective methods for distributing coagulant for mixing and floc formation. In a standard water treatment train, rapid mix occurs between coagulant addition and flocculation, and provides an enhanced method of distributing coagulant evenly through the water. Flocculation is a method to coagulate contaminant particles together for easy removal through sedimentation or filtration, and is an instrumental part of the water treatment train. Our goal is to optimize both treatment processes through modifying parameters such as tank volume, residence time, and flocculator blades, to maximize the amount of contaminant removed through flocculation against maintenance and operation costs.

# Executive Summary 

Regarding both rapid-mix and flocculation we will be using energy dissipation rates as the main constraint to figure out how effectively cogulatant is being distributed in our system as well how effectively the coagulant is forming flocs.

Focusing more specifically on design,  we recommend a pump diffusor as our mechanical rapid-mix mechanism. For mechanical flocculation we will be using rotating blade flocculators. We have not yet determined whether or not we want to implement horizontal or vertical blade flocculators. Regardless of the orientation of the blades, we will compartmentalize the flocculation tank so that a tapered velocity graidnet profile can be imposed. A high to low velocity gradient in subsequent compartments will create dense flocs. It will also reduce the total retention time.

We will be determining the required rapid-mix volumne by it's residence time. The flocculation tank volume will also be termined by it's residence time; however this volume will be more open ended because the residence time is not necessarily constant.

[/Monroe/]: # (Mixing coagulant uniformly with the raw water before splitting the flow into multiple treatment trains might be one of the goals of rapid mix in this plant. I suspect that your design will not be influenced in any way by knowing the contaminant that controls the coagulant dosage because we don't have a physics-based model for the design of the rapid mix. See https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Derivations.html#turbulent-large-scale-eddies for mixing length estimates. Note that you would want to include a safety factor on these equations.)


# Constraints

* Relevant Regulations


* Client Expectations
    * Redundancy plan (at least one backup unit for both rapid mix and floccculation)


* Site and Budget
    * Mechcanical mechanisms require a power input which then adds to operation costs
    * Rapid mix mechanical is one of the most energy intensive units in treatment train
    
* Process Capabilities Including Process Design Equations
    * Residence time will determine the required volume of rapid-mix


    * "Rapid blending must be achieved with water treatment chemicals that involve irreversible competitive consecutive reactions." (MWH) 


* Ease of Use
    * Static mixers are simple and reliable but they are prone to clogging (MWH)

* Operation Requirements
    * If there are multiple treatment trains, parallel treatment trains must receive the same concentration of coagulant; coagulant must be mixed equally if rapid mix units are before flow splitting
    * Type of coagulant used/dosage
    * The optimal pH for aluminum coagulant nanoparticle formation is between pH of 6.5 and 8.5. Adding coagulant usually reduces pH so somtimes it is necessary to add a base to increase the pH of the raw water
    * pH in the 4.4 to 6 range is optimal for flocculation
    * High temperature accelerates the flocculation process (flocculation may be slower in the winter and therefore operators may need to allow for more time when the temp is colder)

* Maintenance Requirements
    * Needs to be easy to clean
    * Parts that can be easily bought and replaced
    * Vertical shaft rotating blades don't require underwater bearings where as horizonal shaft rotating blades do



[/Monroe/]: # (Quote from Benjamin and Lawler [https://newcatalog.library.cornell.edu/catalog/8516026] page 546: "While much research remains to be done on the interactions of chemicals, paricles, and mixing, the emerging picture at this point is tha ta high degree of turbulence leads to more efficient chemical use for destabilization by adsorption and charge neutralization, whereas destabilization by precipitation is less sensitive to the mixing conditions." My understanding is that drinking water treatment plants use destabilization by precipitation; the only mechanism that we have been able to replicate in our labs here at Cornell. We now recognize that it isn't destabilization, but rather is simply that the coagulant nanoparticles are sticky.)

See [new resources](https://raw.githubusercontent.com/MWSteams/4590FSP/master/resources.md) for information on rapid mix and the Cottonwood plant.


# Physics

$$G=\frac{\Delta v}{\Delta h}$$

where $G$ is the velocity gradient, $\Delta v$ is the difference in velocity between two points on two different streamlines and $\Delta h$ is the distance between the streamlines.

G is a measure of how quickly one point of water along one streamline moves in comparison to another point on a different streamline.

$$\varepsilon = G^2 \nu$$ 

where $\epsilon$ is the energy dissipation rate, $G$ is the velocity gradient, and $\nu$ is the kinematic viscosity of water.

It is hypotheiszed that it's ultimately velocity gradients that transport coagulant nanoparticles to the surface of particles.

$$P = \rho \bar G^2 \nu Q \theta$$ 

where $P$ is power input required, $\rho$ is density of water, $Q$ is the flow rate, and $\theta$ is the hydraulic residence time.

This equation can be used to estimate electrical motor sizing requirements for mechanical rapid mix units.

$$\lambda_\nu = \Pi_{K\nu}\left( \frac{\nu^3}{\varepsilon} \right)^{\frac{1}{4}}$$

where $\lambda_\nu$ is the inner viscous length scale, $\Pi_{K\nu}=5-$, $\nu$ is the kinematic viscosity of water, and $\epsilon$ is the energy dissipation rate.

"At length scales larger than the inner viscous length scale the dominant transport mechanism is by turbulent eddies. At length scales smaller than the inner viscous length scale the dominant transport mechanism is fluid deformation due to shear." (AguaClara) This equations strongly suggests that the coagulant nanoparticles are not transported to the clay surfaces in the short amount of time spent in the rapid mix.

[/Monroe/]: # (The previous equation needs to be combined with an estimate of the diffusion length scale for the coagulant nanoparticles. In any case that equation strongly suggests that the coagulant nanoparticles are not transported to the clay surfaces in the short amount of time spent in the rapid mix.)
[/Lynn/]: # (Added)

[/Monroe/]: # (Another equation is the eddy turnover time
https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Derivations.html#equation-rapid-mix-rm-derivations-38)

# Table Listing Potential Strategies to Meet the Constraints and Treatment Objectives

| strategies | advantages | disadvantages | data needed for decision |
| ---------- | ---------- | ---------- | ---------- |
| Reducing energy used to power mechanical mixers | Lower cost, lower electricity usage | Less power to mixers affects energy dissipation, velocity, residence time, and more | Power usage typical for mechanical rapid mix |
| Modifying coagulant dosage to ensure removal of arsenic | Meet water treatment plant standards for arsenic removal | Rapid mix can be expensive (coagulant and electricity to power mixers) | Coagulant dosage needed to remove certain concentration of arsenic, cost of powering rapid mixers
| Modifying coagulant dosage to ensure even distribution | Increased coagulant will remove more contaminant (arsenic) | Increased coagulant will be more expensive | Type of coagulant used, velocity gradient, ideal dosage, residence time of mixer |
|Horizontal Rotating Blades| More simple compartmentalization, different tapered G values are produced for one particular drive shaft rotational speed| Requires underwater bearings| Flocculation performance|
|Vertical Rotating Blades| Do not require underwater bearings| A much wider diversity of G values within the flocculation basin for a given mean velocity gradient| Flocculation performance|
|Flocculator tank size|Larger tanks result in a longer residence time for flocs|Possible increase in maintenance/operation costs for larger tanks| Flocculation performance using tanks of various sizes|


![](https://github.com/MWSteams/4590FSP/raw/Rapid_Mix_Mechanical/pump1.PNG)

[Pumped Flash Mixer]

# Vendors and Contact Information File

[/Monroe/]: # (Excellent work finding some suppliers)

In [0]:
vendors = pd.read_csv('https://raw.githubusercontent.com/MWSteams/4590FSP/master/vendors.txt',sep='\t')
vendors

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
0,plate settlers,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...,4/16/2020,Ian Cullings,NaN
1,sludge removal system,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/products/hoseles...,4/16/2020,Ian Cullings,NaN
2,plate settlers,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,https://www.monroeenvironmental.com/water-and-...,4/16/2020,Ian Cullings,NaN
3,sludge removal system,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,http://www.monroeenvironmental.com/water-and-w...,4/16/2020,Ian Cullings,NaN
4,plate settler with hopper,Parkson,still,pending,https://www.parkson.com/,https://www.parkson.com/products/lamella-ecoflow,4/16/2020,Ian Cullings,Includes sloped sludge hopper in design
5,rapid mixer (mech),Dynamix,still,pending,https://www.dynamixinc.com,https://www.dynamixinc.com/industries/waste-wa...,4/16/2020,Lynn Li,NaN
6,rapid mixer (mech),Meurer Research,still,pending,http://www.meurerresearch.com,http://www.meurerresearch.com/products/mixer/,4/16/2020,Lynn Li,NaN
7,rapid mixer (mech),Philadelphia Mixing Solutions,still,pending,https://www.philamixers.com,https://www.philamixers.com/industries/water-w...,4/16/20,Lynn Li,NaN


# Shared Assumptions

See this [python file, Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py)

Use this file to hold parameters that are in the SOW or elsewhere that
influence the design of the entire plant. This provides an easy mechanism
for all of us to share the same assumptions. Read the SOW to find sections that relate to your part of the design and enter all pertinent variables into [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) and then push those changes to github.

We are creating our model for flocculation on the assumption that the main constituents for removal by flocculation are arsenic and dissolved organic matter. Rapid mixing before flocculation will determine the degree of removal by flocs.

[/Monroe/]: # (Rewrite this now that you recall that rapid mix doesn't remove anything.)

[/Monroe/]: # (You can also create a range of rapid mix volumes, power requirements, and annual operating costs for standard designs for rapid mix units. Those preliminary designs are trivial and will help give insight for goal of reducing the energy footprint of this facility.)

Preliminary assumptions we will be making involve associated costs, such as costs for powering the mixers and operation and maintenance costs for each treatment process. As we begin to develop the code for each process, we will decide on which variables we will be changing and which will be assumed to be fixed.

# Major Decisions

Write several paragraphs describing the major decisions that you will need to make in order to create a design and outline a strategy for how you will obtain the necessary information to make those decisions.

# Next Steps and Challenges

As a next step we will need to:


1) Determine the number of rapid mix and flocculation units that we want to have in place. For now we will use the random constants given in the Fairmont sheet but this will get updated if we find more accurate ways to determine these values.

2) Determine if we are going to use vertical vs horizontal blades.

3) Find vendors use specific stats from these vendors to continue our quantitative analysis.

Hopefully we will be able to find some case studies that include quantitative data in order to move our deisgn forward. We will also be looking at previous AguaClara rapid mix and flocculation mechanical designs.

# Design

Begin writing the code to start your design. Define flow rates, mass rates, energy requirements, and any readily calculated dimensions, velocities, flows, etc.

In [6]:
## Givens
# print('The average flow to be treated at Fairmont is',fmt.q_ave.to(u.Mgal/u.day),'or')
# print('The maximum flow to be treated at Fairmont is',fmt.q_max.to(u.Mgal/u.day))
specific_energy = (fmt.electricity_power/fmt.q_ave).to(u.J/u.L)
temp = 15 * u.degC
specific_potential_energy = (specific_energy / u.gravity / ac.density_water(temp)).to(u.m)
fsp_flow = 480*(u.Mgal/u.day)

print('The average amount of energy used per liter of water is expected to be',specific_energy,'or',specific_potential_energy,'of potential energy')

# Adjusting pH (operating at an optimum pOH mitigates the need to shift optimum pH based on temp)
#pkW = (0.01760*temp.to(u.degK))+(4470.99/temp.to(u.degK))-6.0875
#pH + pOH = pKW

## Constants (random by Monroe)
fmt.n_rm_mech #4 units (one for redunancy)
fmt.L_rm_mech #3m
fmt.W_rm_mech #3m

fmt.n_floc_mech #4 units
fmt.L_floc_mech #10m
fmt.W_floc_mech #10m

#pi = 3.1415926
#Don't redefine constants that are already defined for you. use np.pi
# add kinematic velocity

## Rapid Mix Mechanical
residence_time_rm_mech = 1*u.min
volume_rm_mech = (fsp_flow*residence_time_rm_mech).to(u.gal)
n_rm_mech = fmt.n_rm_mech-1
volume_per_rm_mech = (volume_rm_mech/n_rm_mech).to(u.m**3)
area_per_rm_mech = volume_per_rm_mech/fmt.L_rm_mech
radius_rm_mech_pipe = math.sqrt((area_per_rm_mech/np.pi).magnitude)
radius_rm_mech_pipe = radius_rm_mech_pipe*u.m
diameter_rm_mech_pipe = radius_rm_mech_pipe*2
print(diameter_rm_mech_pipe)

## Flocculation Mechanical
#residence_time_f = volumn_f/fsp_flow


The average amount of energy used per liter of water is expected to be 118.1 joule / liter or 12.05 meter of potential energy
13.36 meter


# References

https://aguaclara.github.io/Textbook/Rapid_Mix

https://education.seattlepi.com/flocculation-wastewater-4866.html

https://www.iwapublishing.com/news/coagulation-and-flocculation-water-and-wastewater-treatment